# 数据加载、存储与文件格式

读写文本格式数据
-----------

In [1]:
import pandas as pd

In [1]:
!cat ch06/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:

df=pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('ch06/ex1.csv',sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
!cat ch06/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [8]:
pd.read_csv('ch06/ex2.csv',header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
pd.read_csv('ch06/ex2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv('ch06/ex2.csv',names=['a','b','c','d','message'],index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [6]:
!cat ch06/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [9]:
parsed=pd.read_csv('ch06/csv_mindex.csv',index_col=['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [23]:
result=pd.read_table('ch06/ex3.txt',sep='\s+')

In [24]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [26]:
!cat ch06/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [28]:
pd.read_csv('ch06/ex4.csv',skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [30]:
!cat ch06/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [34]:
result=pd.read_table('ch06/ex5.csv',sep=',')
result

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [37]:
result.isnull()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [39]:
result=pd.read_csv('ch06/ex5.csv',na_values=['NULL'])

In [41]:
result

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [43]:
sentinels={'message':['foo','NA'],'something':['two']}
pd.read_csv('ch06/ex5.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11,12,NaN


逐块读取文本文件
------------

In [46]:
result=pd.read_csv('ch06/ex6.csv')

In [49]:
pd.read_csv('ch06/ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [52]:
chunker=pd.read_csv('ch06/ex6.csv',chunksize=1000)
chunker

In [61]:
chunker=pd.read_csv('ch06/ex6.csv',chunksize=1000)
tot=pd.Series([])
for piece in chunker:
    tot=tot.add(piece['key'].value_counts(),fill_value=0)
tot=tot.sort_values(ascending=False)
tot

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
V    328
I    327
U    326
P    324
D    320
A    320
R    318
Y    314
G    308
S    308
N    306
W    305
T    304
B    302
Z    288
C    286
4    171
6    166
7    164
8    162
3    162
5    157
2    152
0    151
9    150
1    146
dtype: float64

将数据写出到文本格式
----------------

In [63]:
data=pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [65]:
data.to_csv('ch06/out.csv')

In [67]:
!cat ch06/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [73]:
import sys
data.to_csv(sys.stdout,sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [78]:
data.to_csv(sys.stdout,na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [80]:
data.to_csv(sys.stdout,index=False,header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [85]:
data.to_csv(sys.stdout,index=False,cols=['a','b'])

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [91]:
import numpy as np
dates=pd.date_range('1/1/2000',periods=7)
ts=pd.Series(np.arange(7),index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64

In [93]:
ts.to_csv('ch06/tsseries.csv')

In [95]:
!cat ch06/tsseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [97]:
pd.Series.from_csv('ch06/tseries.csv',parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

手工处理分隔符格式
----------

In [3]:
!cat ch06/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [6]:
import csv
f=open('ch06/ex7.csv')
reader=csv.reader(f)
for line in reader:
    print line

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [9]:
lines=list(csv.reader(open('ch06/ex7.csv')))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]

JSON数据
---------

In [25]:
obj="""{"names":"Wes",
                "places_lived":["United States","Spain","German"],
                "pet":null,
                "siblings":[{"names":"Scott","age":25,"pet":"zuko"},{"names":"Katie","age":33,"pet":"Cisco"}]
                }"""


In [27]:

import json
result=json.loads(obj)


In [29]:
result

{u'names': u'Wes',
 u'pet': None,
 u'places_lived': [u'United States', u'Spain', u'German'],
 u'siblings': [{u'age': 25, u'names': u'Scott', u'pet': u'zuko'},
  {u'age': 33, u'names': u'Katie', u'pet': u'Cisco'}]}

In [32]:
asjson=json.dumps(result)
asjson

'{"pet": null, "siblings": [{"pet": "zuko", "age": 25, "names": "Scott"}, {"pet": "Cisco", "age": 33, "names": "Katie"}], "names": "Wes", "places_lived": ["United States", "Spain", "German"]}'

In [42]:
siblings=pd.DataFrame(result['siblings'],columns=['names','age','pet'])
siblings

,names,age,pet
0,Scott,25,zuko
1,Katie,33,Cisco


In [41]:
result['names']

u'Wes'

In [46]:
siblings=pd.DataFrame(result,columns=['names','pet'])
siblings

ValueError: arrays must all be same length

XML和HTML: Web信息收集
---------

In [47]:
from lxml.html import parse
from urllib2 import urlopen
parsed=parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc=parsed.getroot()


In [53]:
links=doc.findall('.//a')
links[15:20]

[<Element a at 0x7fd00b6643c0>,
 <Element a at 0x7fd00b664418>,
 <Element a at 0x7fd00b664470>,
 <Element a at 0x7fd00b6644c8>,
 <Element a at 0x7fd00b664520>]

In [55]:
lnk=links[28]
lnk

<Element a at 0x7fd00b664838>

In [57]:
lnk.get('href')

'https://autos.yahoo.com/'

In [60]:
lnk.text_content()

'Autos'

In [62]:
urls=[lnk.get('href') for lnk in links]

In [64]:
urls

['https://www.yahoo.com/',
 'https://mail.yahoo.com/?.intl=us&.lang=en-US&.src=ym',
 'https://search.yahoo.com/search',
 'http://news.yahoo.com/',
 'http://sports.yahoo.com/',
 'http://finance.yahoo.com/',
 'https://weather.yahoo.com/',
 'https://games.yahoo.com/',
 'https://answers.yahoo.com/',
 'https://screen.yahoo.com/',
 'https://www.flickr.com/',
 'https://mobile.yahoo.com/',
 'http://everything.yahoo.com/',
 'https://www.yahoo.com/politics',
 'https://celebrity.yahoo.com/',
 'https://www.yahoo.com/movies',
 'https://www.yahoo.com/music',
 'https://www.yahoo.com/tv',
 'https://groups.yahoo.com/',
 'https://www.yahoo.com/health',
 'https://www.yahoo.com/style',
 'https://www.yahoo.com/beauty',
 'https://www.yahoo.com/food',
 'https://www.yahoo.com/parenting',
 'https://www.yahoo.com/makers',
 'https://www.yahoo.com/tech',
 'http://shopping.yahoo.com/',
 'https://www.yahoo.com/travel',
 'https://autos.yahoo.com/',
 'https://www.yahoo.com/realestate',
 'https://www.mozilla.org/firef

In [71]:
tables=doc.findall('.//table')
calls=tables[0]

In [73]:
rows=calls.findall('.//tr')

In [76]:
rows

[<Element tr at 0x7fd00b66ce68>]

In [79]:
def _unpack(row,kind='td'):
    elts=row.findall('.//%s'%kind)
    return [val.text_content() for val in elts]
_unpack(rows[1],kind='th')

IndexError: list index out of range

二进制数据格式
--------------

In [82]:
frame=pd.read_csv('ch06/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
